# Training of AlexNet

In [1]:
import os
import numpy as np
import tensorflow as tf
import tf_keras as keras
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt


In [2]:
# parameters
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 6

DATA_DIR = "../dataset_grayscale"
CLASSES = ["Normal", "Lung Opacity"]


In [3]:
images = []
labels = []

for label, cls in enumerate(CLASSES):
    class_dir = os.path.join(DATA_DIR, cls)
    for file in os.listdir(class_dir):
        if file.lower().endswith((".png", ".jpg", ".jpeg")):
            path = os.path.join(class_dir, file)
            img = Image.open(path)
            img = img.resize((IMG_SIZE, IMG_SIZE))
            img = np.array(img) / 255.0
            img = np.expand_dims(img, axis=-1)  # canal unique
            images.append(img)
            labels.append(label)


In [4]:
images = np.array(images, dtype=np.float32)
labels = np.array(labels, dtype=np.int32)

In [ ]:
indices = np.arange(len(images))
np.random.seed(42)
np.random.shuffle(indices)
images = images[indices]
labels = labels[indices]

# split
total = len(images)
train_size = int(0.7 * total)
val_size   = int(0.15 * total)

# train, val, test
x_train = images[:train_size]
y_train = labels[:train_size]
x_val   = images[train_size:train_size+val_size]
y_val   = labels[train_size:train_size+val_size]
x_test  = images[train_size+val_size:]
y_test  = labels[train_size+val_size:]

print("Train:", x_train.shape)
print("Val:", x_val.shape)
print("Test:", x_test.shape)

# datasets
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds   = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds  = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:

# ALEXNET ARCHITECTURE
model = keras.Sequential([
    keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 1)),

    # Conv 1
    keras.layers.Conv2D(96, 11, strides=4, activation="relu"),
    keras.layers.MaxPooling2D(3, strides=2),

    # Conv 2
    keras.layers.Conv2D(256, 5, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(3, strides=2),

    # Conv 3
    keras.layers.Conv2D(384, 3, padding="same", activation="relu"),

    # Conv 4
    keras.layers.Conv2D(384, 3, padding="same", activation="relu"),

    # Conv 5
    keras.layers.Conv2D(256, 3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(3, strides=2),

    # Connected Layers
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation="relu"),
    keras.layers.Dropout(0.5),

    # Output
    keras.layers.Dense(1, activation="sigmoid")
])



In [ ]:
# compilation
optimizer = keras.optimizers.legacy.Adam()
model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=["accuracy", keras.metrics.AUC(name="auc")]
)

model.summary()

In [ ]:
# training
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

In [ ]:
#evaluation
loss, acc, auc = model.evaluate(test_ds)
print("Test Accuracy:", acc)
print("Test AUC:", auc)

In [ ]:
# resukts
y_true = []
y_pred = []

for x, y in test_ds:
    preds = model.predict(x)
    preds = (preds > 0.5).astype(int)
    y_true.extend(y.numpy())
    y_pred.extend(preds.flatten())

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=CLASSES))

In [ ]:
# save
script_dir = os.path.dirname(__file__)
model.save(os.path.join(script_dir, '..', '..', 'models', 'my_alexnet.h5'))

# Densenet

In [ ]:
base_model = keras.applications.DenseNet121(
    weights="imagenet",
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_model.trainable = False

model = keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
# compilation
optimizer = keras.optimizers.legacy.Adam()
model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=["accuracy", keras.metrics.AUC(name="auc")]
)

model.summary()

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

In [ ]:
loss, acc, auc = model.evaluate(test_ds)
print("Test Accuracy:", acc)
print("Test AUC:", auc)

In [ ]:
y_true = []
y_pred = []

for x, y in test_ds:
    preds = model.predict(x)
    preds = (preds > 0.5).astype(int)
    y_true.extend(y.numpy())
    y_pred.extend(preds.flatten())

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=CLASSES))

In [ ]:
model.save(os.path.join(script_dir, '..', '..', 'models', 'my_densenet.h5'))